# todo

- use geopandas to compute area: https://geopandas.org/data_structures.html
- h3 into geopandas?

In [1]:
from h3._cy.unstable_area import cell_area
import h3.api.basic_int as h3

import numpy as np

In [2]:
def hexes_at_res(res):
    cells = h3.get_res0_indexes()
    cells = h3.uncompact(cells, res)

    hexes = cells - h3.get_pentagon_indexes(res)
    
    return hexes

In [3]:
def hexes_at_res(res):
    cells = h3.get_res0_indexes()
    cells = h3.uncompact(cells, res)

    hexes = cells - h3.get_pentagon_indexes(res)
    
    return hexes


def pentagon_area_avg(res):
    return np.mean([cell_area(p) for p in h3.get_pentagon_indexes(res)])


def hexagon_area_avg(res):
    hexes = hexes_at_res(res)

    return np.mean([cell_area(h) for h in hexes])

In [4]:
for i in range(15):
    r = pentagon_area_avg(i)/pentagon_area_avg(i+1)

    print(f'pent_avg({i})/pent_avg({i+1}): {r}')

pent_avg(0)/pent_avg(1): 7.801194728720636
pent_avg(1)/pent_avg(2): 7.309771164758057
pent_avg(2)/pent_avg(3): 7.1144162454781394
pent_avg(3)/pent_avg(4): 7.043939727427411
pent_avg(4)/pent_avg(5): 7.016303106582753
pent_avg(5)/pent_avg(6): 7.0062688612258475
pent_avg(6)/pent_avg(7): 7.002328037884673
pent_avg(7)/pent_avg(8): 7.000895397448314
pent_avg(8)/pent_avg(9): 7.000332537392249
pent_avg(9)/pent_avg(10): 7.00012792974227
pent_avg(10)/pent_avg(11): 7.000047484368409
pent_avg(11)/pent_avg(12): 7.000018294654858
pent_avg(12)/pent_avg(13): 7.000006754900207
pent_avg(13)/pent_avg(14): 7.000002622034447
pent_avg(14)/pent_avg(15): 7.00000089830143


In [5]:
%%time
for i in range(5):
    r = hexagon_area_avg(i)/hexagon_area_avg(i+1)

    print(f'hex_avg({i})/hex_avg({i+1}): {r}')

hex_avg(0)/hex_avg(1): 7.145837994413481
hex_avg(1)/hex_avg(2): 7.025068368556221
hex_avg(2)/hex_avg(3): 7.003851863076998
hex_avg(3)/hex_avg(4): 7.000565467269358
hex_avg(4)/hex_avg(5): 7.0000816405804525
CPU times: user 11 s, sys: 213 ms, total: 11.2 s
Wall time: 11.3 s


In [6]:
%%time
for i in range(6):
    r = hexagon_area_avg(i)/pentagon_area_avg(i)

    print(f'hex_avg({i})/pent_avg({i}): {r}')

hex_avg(0)/pent_avg(0): 1.7006790067775814
hex_avg(1)/pent_avg(1): 1.8566511182161325
hex_avg(2)/pent_avg(2): 1.9318950499753382
hex_avg(3)/pent_avg(3): 1.9623923801930714
hex_avg(4)/pent_avg(4): 1.9745510148103222
hex_avg(5)/pent_avg(5): 1.9791266917525556
CPU times: user 9.68 s, sys: 164 ms, total: 9.85 s
Wall time: 9.9 s


In [7]:
def num_hexagons(res):
    return h3.num_hexagons(res) - 12

def earth_area():
    return sum(map(cell_area, h3.get_res0_indexes()))

def pentagon_area_total(res):
    return sum(map(cell_area, h3.get_pentagon_indexes(res)))

def implied_hex_area_avg(res):
    A = earth_area() - pentagon_area_total(res)
    
    A = A/num_hexagons(res)
    
    return A

In [8]:
implied_hex_area_avg(0)/hexagon_area_avg(0)

0.9999999999999993

In [9]:
implied_hex_area_avg(1)/hexagon_area_avg(1)

0.9999999999651914

In [10]:
implied_hex_area_avg(2)/hexagon_area_avg(2)

0.9999999999999993

# Verify area calculation against another library

In [11]:
%%time
from other_libs import area_sp

res = 2
cells = h3.get_res0_indexes()
cells = h3.uncompact(cells, res)

relative_error = max(
    np.abs(area_sp(h) - cell_area(h))/area_sp(h)
    for h in cells
)

print(relative_error)

1.0474325063587727e-08
CPU times: user 5.65 s, sys: 273 ms, total: 5.92 s
Wall time: 5.86 s
